<a href="https://colab.research.google.com/github/Vinh-thuy/data_valorization/blob/main/demo/demo_kpe_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install keybert keyphrase-vectorizers flair nlpcloud --quiet

In [ ]:
!python -m spacy download fr_core_news_sm 

In [3]:
from keybert import KeyBERT
from keyphrase_vectorizers import KeyphraseCountVectorizer
from flair.embeddings import TransformerDocumentEmbeddings
import pandas as pd
import nlpcloud
import spacy
from spacy.lang.fr.examples import sentences 
nlp = spacy.load("fr_core_news_sm")

import numpy as np

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [4]:
#@title Connexion Google Drive {display-mode: "form"}

from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

%cd /content/gdrive/MyDrive/P4T/google_colab/demo/


Mounted at /content/gdrive/
/content/gdrive/MyDrive/P4T/google_colab/demo


In [5]:
#@title Construction de la connexion à la base PostgreSQL {display-mode: "form"}

from postgres import postgresQuery
from utilities import DB_HOSTNAME,DB_PORT, DB_NAME, DB_USER, DB_PWD, PG_COL_TO_CLEAN_TABLE_RESTAURANT
pg = postgresQuery(DB_HOSTNAME, DB_PORT, DB_NAME, DB_USER, DB_PWD)

In [6]:
#@title Load des résultats du 1er filtrage simple (distance / horaire / categorie) depuis Google Drive {display-mode: "form"}

import pandas as pd 
res_bulk_distance_horaire_category=pd.read_csv('/content/gdrive/MyDrive/P4T/google_colab/demo/data.csv')  

# Liste des restaurants issue des 3 filtrages
liste_selection_restaurant = res_bulk_distance_horaire_category.restaurant_id.unique()

In [10]:
#@title Récupération des tables : restaurant concat review avec filtrage des restaurant_id issus des 3 filtrages : liste_review_restaurant{display-mode: "form"}
city = 'Paris'

#Cas ou on utilise une liste aleatoire de restaurant pour tests
sql_query_1 = "select r1.*, r2.name as restaurant_name, r2.adress_street as restaurant_adress_street, r2.adress_city as restaurant_adress_city, r2.rating as restaurant_rating, r2.cid as restaurant_cid, r2.category as restaurant_category, r2.reviews_count as restaurant_reviews_count, r2.img_name as restaurant_img_name, r2.place_id as restaurant_place_id  from review r1, restaurant r2  where r1.restaurant_id > 1150 and r1.restaurant_id = r2.restaurant_id and r2.adress_city  = '" + city + "'"
liste_review_restaurant = pg.query_db(sql_query_1)
liste_review_restaurant['review_datetime_utc'] = pd.to_datetime(liste_review_restaurant['review_datetime_utc']).dt.normalize()
liste_review_restaurant['created_at'] = pd.to_datetime(liste_review_restaurant['created_at']).dt.normalize()


#Cas ou on utilise le filtrage simple
#Dataframe avec filtrage des restaurants issus des 3 niveaux de filtrage
#sql_query_1 = "select r1.*, r2.name as restaurant_name, r2.adress_street as restaurant_adress_street, r2.adress_city as restaurant_adress_city, r2.rating as restaurant_rating, r2.cid as restaurant_cid, r2.category as restaurant_category, r2.reviews_count as restaurant_reviews_count, r2.img_name as restaurant_img_name, r2.place_id as restaurant_place_id  from review r1, restaurant r2  where r1.restaurant_id = r2.restaurant_id and r2.adress_city  = '" + city + "'"
#liste_review_restaurant = pg.query_db(sql_query_1)
#liste_review_restaurant['review_datetime_utc'] = pd.to_datetime(liste_review_restaurant['review_datetime_utc']).dt.normalize()
#liste_review_restaurant['created_at'] = pd.to_datetime(liste_review_restaurant['created_at']).dt.normalize()
#liste_review_restaurant = liste_review_restaurant[liste_review_restaurant.restaurant_id.isin(liste_selection_restaurant)]




In [11]:
#@title Création de 3 catégories d'avis : 5 ; 5 +4 ; 1 + 2 {display-mode: "form"}

# On prend pour les categories de Review positifs que ceux dont le tga isLocalGuide est True
avis_pos_5 = liste_review_restaurant.loc[(liste_review_restaurant.rating> 4.0) & (liste_review_restaurant.islocalguide == 'True')]
avis_pos_4_5 = liste_review_restaurant.loc[(liste_review_restaurant.rating> 3.0) & (liste_review_restaurant.islocalguide == 'True')]
avis_neg_1_2 = liste_review_restaurant.loc[(liste_review_restaurant.rating< 3.0)]

avis_pos_5 = avis_pos_5.sort_values(by=['review_datetime_utc'], ascending=False)
avis_pos_4_5 = avis_pos_4_5.sort_values(by=['review_datetime_utc'], ascending=False)
avis_neg_1_2 = avis_neg_1_2.sort_values(by=['review_datetime_utc'], ascending=False)

In [12]:
#@title DataPrep des Reviews : Suppression des versions non FR ou None {display-mode: "form"}

# Suppression des versions non FR = 4_5
df = avis_pos_4_5[avis_pos_4_5['reviews_data'].str.contains("(Traduit par Google)")]

for i, (data, id) in enumerate(zip(df.reviews_data, df.review_id)):
  avis_pos_4_5.loc[avis_pos_4_5["review_id"] == id, "reviews_data"] = data.split('(Traduit par Google)')[1]

# Suppression des versions non FR = 1_2
df = avis_neg_1_2[avis_neg_1_2['reviews_data'].str.contains("(Traduit par Google)")]

for i, (data, id) in enumerate(zip(df.reviews_data, df.review_id)):
  avis_neg_1_2.loc[avis_neg_1_2["review_id"] == id, "reviews_data"] = data.split('(Traduit par Google)')[1]  

# Suppression des None
avis_pos_4_5 = avis_pos_4_5[avis_pos_4_5.reviews_data != 'None']
avis_neg_1_2 = avis_neg_1_2[avis_neg_1_2.reviews_data != 'None']

## NLP CLOUD - QUESTION ANSWER for REVIEWS and QUERIES

In [ ]:
question_positif = [""" liste moi les points positifs de ce restaurant : 
- avec un tri des avis les plus positifs en premier 
- avec les avis disponibles
- avec en séparateur entre chaque point le point-virgule
- sans générer de nouveau texte
- de facon completement objectif"""]

In [ ]:
question_negatif = [""" liste moi les points négatifs de ce restaurant : 
- avec un tri des avis les plus négatifs en premier 
- avec les avis disponibles
- avec en séparateur entre chaque point le point-virgule
- sans générer de nouveau texte
- de facon completement objectif"""]

In [ ]:
def split_Reviews(df, nb_reviews):
  # Création d'une nouvelle colonne : review_len Pour avoir le nb de caractere par reviews_data
  df['review_len'] = ''

  for i, (elt, id) in enumerate(zip(df.reviews_data, df.review_id)):
    df.loc[df["review_id"] == id, "review_len"] = len(elt)



  # On split le Dataframe en entrée par restaurant_id
  dfList = []
  for i, elt in enumerate(df.restaurant_id.unique()):
    dfList.append(df[df.restaurant_id == elt])  


  # Pour chaque bloc : restaurant_id on ne garde que : nb_reviews les plus volumineux en caractere 
  for i in range(len(dfList)):
    dfList[i] = dfList[i].sort_values(by=['review_len'], ascending=False)[:nb_reviews]

  return dfList

In [ ]:
avis_pos_4_5_split = split_Reviews(avis_pos_4_5, 50)
avis_neg_1_2 = split_Reviews(avis_neg_1_2, 50)

In [ ]:
def gpt_extract(client, question, df,review_summary, review_category):
  import re
  import datetime
  import time


  for (reviews_data, id) in zip(df.reviews_data, df.review_id):
    answer = client.question(question, reviews_data)
    text =answer['answer']
    text = re.sub(r'\n\d', ' ', text).replace("\nspoilers","").replace("\n","")
    lst = text.split(";") 

    date_now = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    new_row = {'review_id':id, "keyphrase_extract_review":lst, "review_category":review_category, "created_at":date_now}

    #append row to the dataframe
    review_summary = review_summary.append(new_row, ignore_index=True)
    time.sleep(30)

  return review_summary

In [ ]:
client = nlpcloud.Client("finetuned-gpt-neox-20b", "1cc13406650e4690596159d13c312538b2e8346f", gpu=True, lang="en")

review_summary = pd.DataFrame()
review_summary['keyphrase_extract_review'] = ''
review_summary['review_id'] = ''
review_summary['review_category'] = ''
review_summary['created_at'] = ''

#review_summary = gpt_extract(client, question_positif[0], avis_pos_4_5,review_summary, 'review_positif')

#review_summary = gpt_extract(client, question_negatif[0], avis_neg_1_2,review_summary, 'review_negatif')

In [ ]:
import pandas as pd

print(len(avis_pos_4_5_split))

# On analyse et filtre par la distance pour chaque Bloc : dfList 
for i in range(len(avis_pos_4_5_split)):    
  print(i)
  review_summary = gpt_extract(client, question_positif[0], avis_pos_4_5_split[i],review_summary, 'review_positif')

20
0


KeyboardInterrupt: ignored

In [ ]:
import pandas as pd

# step est la taille de chaque Bulk
step = 50

# On coupe le Dataframe initial en n bloc de dimension = Step
dfList = []
for i in range(0, avis_pos_4_5.shape[0], step):
  dfList.append(avis_pos_4_5[i:i+step])

print(len(dfList))

# On analyse et filtre par la distance pour chaque Bloc : dfList 
for i in range(len(dfList)):    
  print(i)
  review_summary = gpt_extract(client, question_positif[0], dfList[i],review_summary, 'review_positif')

In [ ]:
review_summary['keyphrase_extract_review'].iloc[1]

In [ ]:
# Export des résultats dans Google Drive

review_summary.to_csv('review_summary_split.csv')
!cp data.csv "/content/gdrive/MyDrive/P4T/google_colab/demo/"

In [ ]:
review_split_res = split_Reviews(avis_pos_4_5, 6)

In [ ]:
review_summary

## NLP CLOUD - ENTITIES

In [ ]:
def nlp_cloud_synthese(reviews_data, restaurant_id, cat_reviews):
    client = nlpcloud.Client("finetuned-gpt-neox-20b", "1cc13406650e4690596159d13c312538b2e8346f", gpu=True, lang="fr")

    #Création des résumés à partir des avis 
    #output : openAI obj
    summary_reviews = []

    for i in range(len(reviews_data)):
        summary_reviews.append(client.summarization(reviews_data[i]))
        time.sleep(3)

        
    #Extraction des résumes des avis 
    #output : list
    summary_reviews_2 = []
    for i in range(len(summary_reviews)):
        summary_reviews_2.append(summary_reviews[i]['summary_text'])

    dtedr.create_pickle(summary_reviews_2, restaurant_id + '_' + cat_reviews +'_summary_reviews_2_nlp')

    # la taille totale des résumés des avis positifs est de :
    print("La taille totale résumés des avis est de : "+ str(len(' '.join(str(e) for e in summary_reviews_2))))

    
    
    #Split des résumés des avis par batch de 990 caractères
    #output : list
    my_str = ' '.join(str(e) for e in summary_reviews_2)
    n = 990
    summary_reviews_2bis = [my_str[idx:idx + n] for idx in range(0, len(my_str), n)]


          
    #Création des résumés des résumés des avis (Pass 2)
    #output : list
    summary_reviews_pass2 = []

    for i in range(len(summary_reviews_2bis)):
        summary_reviews_pass2.append(client.summarization(summary_reviews_2bis[i]))
        time.sleep(3)
        
        
    #Extraction des résumés de résumés des avis
    #output : list
    summary_reviews_3 = []
    for i in range(len(summary_reviews_pass2)):
        summary_reviews_3.append(summary_reviews_pass2[i]['summary_text'])

          
    dtedr.create_pickle(summary_reviews_3, restaurant_id + '_' + cat_reviews +'_summary_reviews_3_nlp')

          
    # la taille totale des résumés des résumés des avis positifs est de : :
    print("la taille totale des résumés des résumés des avis est de : "+ str(len(' '.join(str(e) for e in summary_reviews_2))))
    print(' ')
    print(' '.join(str(e) for e in summary_reviews_3))
    
    return summary_reviews_3


In [ ]:
client = nlpcloud.Client("finetuned-gpt-neox-20b", "1cc13406650e4690596159d13c312538b2e8346f", gpu=True, lang="en")

In [ ]:
searched_entity="points positifs"

In [ ]:
searched_entity="plat à tester"

In [ ]:
searched_entity="ce qui est apprécié"

In [ ]:
searched_entity="points negatifs"

In [ ]:
searched_entity="probleme"

In [ ]:
searched_entity="probleme"

In [ ]:
searched_entity_list

In [ ]:
searched_entity_list = ["points positifs", "plat à tester", "ce qui est apprécié", "qualité du service"]

In [ ]:
searched_entity_list = ["points negatifs", "ce qui n'est pas apprécié", "avis critiques", "probleme"]

In [ ]:
searched_entity_list = ["ce qui est recherché", "localisation du restaurant", "critere du restaurant"]

In [ ]:
text2KPE = reviews12[0]

In [ ]:
text2KPE = querie1[0]

In [ ]:
text2KPE

In [ ]:
import time

list_res_search = []

for search in enumerate(searched_entity_list):
  res_nlpCloud = client.entities(text2KPE, searched_entity=search[1])
  list_res_i = json_extract(res_nlpCloud['entities'],'text')
  list_res_search = list_res_search + list_res_i
  time.sleep(1)

list_res_search = list(dict.fromkeys(list_res_search))
list_res_search

## DATAPREP DU RESULTATS DES KEYPHRASE

In [ ]:
def print_spacy(kpe):  
  for i in range(len(kpe)):
    texte = kpe[i][0]
    #print_spacy(texte)

    # Initialising doc with a sentence.
    doc = nlp(texte)

    # Using properties of token i.e. Part of Speech and Lemmatization
    for token in doc:
      print(token, " | ", spacy.explain(token.pos_), " | ", token.lemma_)
    
    print("---------------------------------------------------------")
    

In [ ]:

def identify_row_to_delete(texte):  
  # Initialising doc with a sentence.
  doc = nlp(texte)

  suppression = False

  test_list = ['adposition','determiner','pronoun', 'verb', 'subordinating conjunction']

  if len(doc)== 1:
    token = doc[0]  
    spacy_string = str(spacy.explain(token.pos_))
    suppression = [ele for ele in test_list if(ele == spacy_string)]
    suppression = bool(suppression)
    #print(suppression)

  if len(doc)== 2:
    token1 = doc[0]  
    token2 = doc[1]  
    if str(spacy.explain(token1.pos_)) == "pronoun" and str(spacy.explain(token2.pos_)) == "verb":
      suppression = True

  return suppression



def suppression_word(texte):
  test_word = ['coordinating conjunction']
  doc = nlp(texte)
  list_sup = []

  for idx, element in enumerate(doc):
    if [ele for ele in test_word if(ele == str(spacy.explain(element.pos_)))] and idx == 0:
      #print(idx)
      #print(spacy.explain(element.pos_))
      list_sup.append(idx)
      #doc = remove_span(doc, idx)

    if [ele for ele in test_word if(ele == str(spacy.explain(element.pos_)))] and idx == len(doc)-1:
      #print(idx)
      #print(spacy.explain(element.pos_))
      list_sup.append(idx)
      #doc = remove_span(doc, idx)
      
  doc = [doc[i] for i in range(0, len(doc)) if i not in list_sup]
  resultat = ' '.join(str(x) for x in doc)

  return resultat




def suppression_row_word(kpe):
  list_sup_row = []
  list_sup_word = []

  for i in range(len(kpe)):
    texte = kpe[i][0]

    if identify_row_to_delete(texte):
      list_sup_row.append(i)

    new_texte = suppression_word(texte)

    #print(new_texte)
    #print(kpe[i][1])
    kpe[i] = (new_texte, kpe[i][1])


  kpe = [kpe[i] for i in range(0, len(kpe)) if i not in list_sup_row]

  return kpe
  
  

In [ ]:
kpe2

In [ ]:
print_spacy(kpe2)

In [ ]:
res3 = suppression_row_word(kpe2)
res3

In [ ]:
kpe2

In [ ]:
test_word = ['coordinating conjunction']
texte = "et avons dégustez ou"
doc = nlp(texte)
#print(len(doc))

list_sup = []

for idx, element in enumerate(doc):
  if [ele for ele in test_word if(ele == str(spacy.explain(element.pos_)))] and idx == 0:
    print(idx)
    print(spacy.explain(element.pos_))
    list_sup.append(idx)
    #doc = remove_span(doc, idx)

  if [ele for ele in test_word if(ele == str(spacy.explain(element.pos_)))] and idx == len(doc)-1:
    print(idx)
    print(spacy.explain(element.pos_))
    list_sup.append(idx)
    #doc = remove_span(doc, idx)

#doc = [doc[i] for i in range(0, len(doc)) if i not in list_sup]
 

In [ ]:

texte = "et avons dégustez"
doc = nlp(texte)

test_word = ['coordinating conjunction']

for idx, element in enumerate(doc):
  if [ele for ele in test_list if(ele == str(spacy.explain(element.pos_)))] and idx == 0:
    print(idx)
    print(spacy.explain(element.pos_))


print(doc)
test_word = ['coordinating conjunction']
toto = [doc[i] for i in range(0, len(doc)) if i not in test_word]
toto

## happytransformer

In [ ]:
pip install happytransformer

In [ ]:
from happytransformer import  HappyTextToText

In [ ]:
from datasets import load_dataset

In [ ]:
happy_tt = HappyTextToText("T5", "PoloHuggingface/French_grammar_error_corrector")

In [ ]:
from happytransformer import TTSettings

beam_settings =  TTSettings(num_beams=20, min_length=1, max_length=100)

In [ ]:
input_text_1 = "grammar: élevé que la moyenne mais"

In [ ]:
output_text_1 = happy_tt.generate_text(input_text_1, args=beam_settings)
print(output_text_1.text)

Plus élevé que la moyenne, mais.


## DATAPREP AVEC SPACY AVANT KEYPHRASE EXTRACTION
**non utilisé en final**

In [ ]:

import spacy
print('spaCy Version: %s' % (spacy.__version__))
spacy_nlp = spacy.load('fr_core_news_sm')


article = "super spot pour manger au marché"
doc = spacy_nlp(article)
doc

res = answer['answer']
for i in res:
  doc = spacy_nlp(i[0])
  print(i[0])
  tokens = [token.text for token in doc if not token.is_stop]
  print(tokens)

  import spacy
nlp = spacy.load('fr_core_news_sm')

# Tokenize words so they can be individually looped through
docs2 = list(nlp.pipe(yourdataframe['yourtextcolumn'].apply(str)))

# Iterate over parsed document objects 
for doc in res:
  	# remove stop words by only keeping non-stop words
    tokens = [token for token in doc if not token.is_stop]
    

### CONCATENATION DES KEYPHRASE EN UNE SENTENCE

In [ ]:
sentence = ""
for count, i in enumerate(res):
  if count == 0:
    sentence = i[0]
  else:
    sentence = sentence + "','" + i[0]

In [ ]:
sentence

"super spot pour manger au marché','très bien assaisonnés','un velouté de champignons très bon','bons produits','ultra chaleureux','le service','vin très bon et','très original avec','nous avons','les"

### COSINE SIMILARITY

### Solution 1

In [ ]:
pip install -U sentence-transformers


In [ ]:
# Fonction utilisant le Transformer HugginFace
def embedding_huggingFace_transformers(string):
  from transformers import AutoTokenizer, AutoModel
  import torch
  import torch.nn.functional as F

  #Mean Pooling - Take attention mask into account for correct averaging
  def mean_pooling(model_output, attention_mask):
      token_embeddings = model_output[0] #First element of model_output contains all token embeddings
      input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
      return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


  # Sentences we want sentence embeddings for
  sentences = string

  # Load model from HuggingFace Hub
  tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')
  model = AutoModel.from_pretrained('sentence-transformers/all-mpnet-base-v2')

  # Tokenize sentences
  encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

  # Compute token embeddings
  with torch.no_grad():
      model_output = model(**encoded_input)

  # Perform pooling
  sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

  # Normalize embeddings
  sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

  #print("Sentence embeddings:")
  #print(sentence_embeddings)
  return sentence_embeddings

In [ ]:
# Fonction utilisant le Sentence Transformer
def embedding_sentence_transformers(string):
  from sentence_transformers import SentenceTransformer
  
  model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
  embeddings = model.encode(string)
  return embeddings

In [ ]:

res_embedding_sentence_transformers = embedding_sentence_transformers(sentence)
res_embedding_huggingFace_transformers = embedding_huggingFace_transformers(sentence)

NameError: ignored

In [ ]:
query = "je cherche un restaurant situé a proximité du marché"
queries = [query]

In [ ]:
query_embedding = embedding_sentence_transformers(query)
query_embeddings = [query_embedding]
#query_embeddings

In [ ]:
!pip install scipy
import scipy

In [ ]:
reviews_embedding = res_embedding_sentence_transformers

In [ ]:
print("Semantic Search Results")

number_top_matches = 5
for query, query_embedding in zip(queries, query_embeddings):
    distances = scipy.spatial.distance.cdist([query_embedding], [reviews_embedding], "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for idx, distance in results[0:number_top_matches]:
        print(sentence, "(Cosine Score: %.4f)" % (1-distance))

Semantic Search Results


NameError: ignored

### Solution 2

In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

query_embedding = model.encode('How big is London')
passage_embedding = model.encode(['London has 9,787,426 inhabitants at the 2011 census',
                                  'London is known for its finacial district'])

print("Similarity:", util.dot_score(query_embedding, passage_embedding))

Downloading:   0%|          | 0.00/737 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/11.5k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/383 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.8k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

Similarity: tensor([[0.5472, 0.6330]])


### Solution 3

In [ ]:
"""
This is a simple application for sentence embeddings: semantic search

We have a corpus with various sentences. Then, for a given query sentence,
we want to find the most similar sentence in this corpus.

This script outputs for various queries the top 5 most similar sentences in the corpus.
"""
from sentence_transformers import SentenceTransformer, util
import torch

#embedder = SentenceTransformer('all-MiniLM-L6-v2')
embedder = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# Corpus with example sentences
corpus = ['super spot pour manger au marché',
          'le burger est excellent',
          'la serveuse est désagréable',
          'les pates et les spaghettis sont tres bonnes',
          'ultra chaleureux',
          'le service exceptionnel',
          'le lieu est tres bruyant',
          'restaurant reposant',
          'nous avons'
          ]

#corpus = [sentence]

corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

# Query sentences:
queries = ["je cherche un restaurant situé a proximité du marché", 'je cherche un restaurant italien', 'je cherche un burger']
queries = list_res_search

# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = min(5, len(corpus))
for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    # We use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1]):
        print(corpus[idx], "(Score: {:.4f})".format(score))

    """
    # Alternatively, we can also use util.semantic_search to perform cosine similarty + topk
    hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=5)
    hits = hits[0]      #Get the hits for the first query
    for hit in hits:
        print(corpus[hit['corpus_id']], "(Score: {:.4f})".format(hit['score']))
    """





Query: restaurant sans bruit et au calme

Top 5 most similar sentences in corpus:
le lieu est tres bruyant (Score: 0.4842)
la serveuse est désagréable (Score: 0.4718)
le burger est excellent (Score: 0.4582)
les pates et les spaghettis sont tres bonnes (Score: 0.4582)
super spot pour manger au marché (Score: 0.4515)




Query: service de qualité

Top 5 most similar sentences in corpus:
le service exceptionnel (Score: 0.6310)
la serveuse est désagréable (Score: 0.5493)
super spot pour manger au marché (Score: 0.3691)
le lieu est tres bruyant (Score: 0.3672)
nous avons (Score: 0.3517)




Query: plats italien ou des burgers

Top 5 most similar sentences in corpus:
le burger est excellent (Score: 0.6444)
les pates et les spaghettis sont tres bonnes (Score: 0.5730)
la serveuse est désagréable (Score: 0.4564)
nous avons (Score: 0.4411)
super spot pour manger au marché (Score: 0.4299)




Query: sans bruit

Top 5 most similar sentences in corpus:
le lieu est tres bruyant (Score: 0.5412)
u